In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain, LLMRouterChain
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate

In [3]:
beginner_template = '''You're a physics teacher who is really focused on 
beginners and explainning complex concepts in simple to understand terms.
You assume no prior knowledge. Here is your question :\n {input}'''

In [4]:
expert_template = '''You're a physics professor who explains physics topics
to advanced audiance members. You can assume anyone you answer has a Phd in Physics.
Here is you question :\n {input}'''

In [5]:
#ROUTE PROMT INFORMATION

prompt_info = [
    {'name':'beginner physics',
     'description':'Answers Basic Physics Questions',
     'template':beginner_template,},

     {'name':'advanced physics',
     'description':'Answers advanced physics questions',
     'template':expert_template,}
]

In [6]:
llm = ChatOpenAI(api_key= api_key)

destination_chains = {}

for p_info in prompt_info:
    name = p_info['name']
    description = p_info['description']
    prompt_template = p_info['template']
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt = prompt)
    destination_chains[name] = chain

c:\Users\Suriya\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
default_prompt = ChatPromptTemplate.from_template('{input}')
default_chain = LLMChain(llm=llm, prompt = default_prompt)

In [8]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [9]:
destinations = [f"{p['name']} : {p['description']}"  for p in prompt_info]

In [10]:
destinations

['beginner physics : Answers Basic Physics Questions',
 'advanced physics : Answers advanced physics questions']

In [11]:
destination_str = "\n".join(destinations)

In [12]:
print(destination_str)

beginner physics : Answers Basic Physics Questions
advanced physics : Answers advanced physics questions


In [13]:
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

In [14]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations = destination_str)

In [15]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
beginner physics : Answers Basic Physics Questions
advanced physics : Answers advanced physics questi